In [ ]:
sub evaluate_algorithm_train_test_split{
    my ($self, $dataset, $algorithm, %args) = ((splice @_, 0, 3), split=>0.6, metric=>undef, @_);

    my ($train, $test) = sml->train_test_split($dataset, split=>args{split});
    my ($actual, $predicted, $score);
    my $test_set = $test->copy();
    # (my slice = $test_set->slice('X', -1)) .= mx->nd->full([$test_set->len, 1], 'NaN);

    $predicted = $algorithm->('sml', $train,, $test_set, @_);
    $actual    = $test->slice_axis(axis=>1, begin=>-1, end=>$test->shape->[1])->squeeze();

    #Regression : Classification
    if (defined $args{metric}){
        if ($args{metric} =~ /accuracy/i){
            $score = sml->accuracy_metric($actual, $predicted);
        }elsif($args{metric} =~ /rmse/i){
            $score = sml->rmse_metric($actual, $predicted);
        }
    }else{
        if(mx->nd->sum($actual->trunc() - $actual) ->asscalar != 0){
            $score = sml->rmse_metric($actual, $predicted);
        }else{
            $score = sml->accuracy_metric($actual, $predicted);
        }
    }

    return wantarray ? ($score, $train, $test, $actual, $predicted) : $score;
}